In [ ]:
if 'google.colab' in str(get_ipython()):
    import sys
    from google.colab import drive, output
    drive.mount('/content/drive')
    !pip instalfrom torch.utils.data import Datasetl torchaudio
    !pip install wandb --upgrade
    # !wandb login
    !pip install umap-learn
    !pip install umap-learn[plot]
    !pip install holoviews
    !pip install -U ipykernel
    !pip install joypy
    # !pip install umap-learn
    output.clear()
    print("Running on colab")
    %load_ext autoreload
    %autoreload 1
    %cd '/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project'
    #sys.path.append('/content/drive/MyDrive/Deep Learning/AutoEncoders/Project/VQVAE_Working/data')
    #sys.path.append('/content/drive/MyDrive/Deep Learning/AutoEncoders/Project/VQVAE_Working/models')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Dataloader')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Models')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Modules')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Clustering_Results/Results')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Clustering_Results/Figures')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Clustering_Result')
elif "zmqshell" in str(get_ipython()):
    print("Running on MIRP")
    root = "/home/mirp_ai/Documents/Daniel_Nieto/PhD/AutoEncoders_Ecoacoustics/Zamuro"
    root_path = "/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido"
else:
    import pathlib
    temp = pathlib.PosixPath
    pathlib.PosixPath = pathlib.WindowsPath
    print("Running local")

In [ ]:
import numpy as np
import torchaudio
import torch
import os, sys
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd

!pip install torchvggish==0.1
from torchvggish import vggish, vggish_input

# % cd f"{root}/Jaguas/vggish"
# folders = os.listdir(root_path)
# files=[]
# for i in range(len(folders)):
#     path_aux = "{}/{}".format(root_path, folders[i])
#     files += list(Path(path_aux).rglob("*.{}".format("wav")))
# file = str(files[16])

In [ ]:
def find_pickle_filenames( path_to_dir, suffix=".pickle" ):
    filenames = os.listdir(path_to_dir)
    return sorted([ filename for filename in filenames if filename.endswith( suffix ) ])

def find_csv_filenames( path_to_dir, suffix=".csv" ):
    filenames = os.listdir(path_to_dir)
    return sorted([ filename for filename in filenames if filename.endswith( suffix ) ])

def find_wav_filenames( path_to_dir, suffix=".wav" ):
    filenames = os.listdir(path_to_dir)
    return sorted([ filename for filename in filenames if filename.endswith( suffix ) ])

folders = os.listdir(root_path)

In [ ]:
root_audios = f"{root}/Complementary_Files/Audios_Zamuro/zamuro_audios.csv"
df_audios = pd.read_csv(root_audios, index_col="Unnamed: 0")
df_audios

In [ ]:
i=0

file = root_path+"/"+df_audios.iloc[i].loc["field_number_PR"]+"/"+df_audios.loc[i, "name_FI"]+".WAV"
audio_sample = vggish_input.wavfile_to_examples(file)
embeddings = embedding_model.forward(audio_sample)
embeddings.shape

In [ ]:
X = []
y = []
for i in range(len(df_audios)):
    embedding_model = vggish()
#     embedding_model = embedding_model.to("cuda")
    embedding_model.eval()
    if (i%100) == 0:
        print(i)
    else:
        pass
    try:
        file = root_path+"/"+df_audios.iloc[i].loc["field_number_PR"]+"/"+df_audios.loc[i, "name_FI"]+".WAV"
        audio_sample = vggish_input.wavfile_to_examples(file)
        embeddings = embedding_model.forward(audio_sample)
    except:
        print(f"Error in {file}")
        continue
    X.append(embeddings.detach())
    y.append(file)
torch.save(X, f"Features_vggish/vggish_features_2.pth")
torch.save(y, f"Features_vggish/vggish_filenames_2.pth")

In [ ]:
X=torch.load(f"Features_vggish/vggish_features.pth", map_location="cpu")
y=

In [ ]:
X[0].shape

In [ ]:
for i in range(len(X)):
    X[i] = X[i].reshape(X[i].shape[0]*X[i].shape[1])

In [ ]:
# Initialise model and download weights
embedding_model = vggish()
embedding_model.eval()

example = vggish_input.wavfile_to_examples(root_path+"/"+folders[0]+"/"+df_audios.loc[0, "Filename"])
embeddings = embedding_model.forward(example)
embeddings.detach().numpy().shape
plt.imshow(embeddings.detach().numpy())

In [ ]:
from torch.utils.data import DataLoader
root = "/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project"
dataset_test = torch.load(f'{root}/temporal/dataset_test_ae_jaguas_new.pth')
dataset_train = torch.load(f'{root}/temporal/dataset_train_ae_jaguas_new.pth')
training_loader = DataLoader(dataset_train, batch_size=1)
test_loader = DataLoader(dataset_test, batch_size=1)
iterator = iter(training_loader)

training_recorder_list = []
training_hour_list = []
training_minute_list = []
delete_samples = []

In [ ]:
def fileToRecord(file, audio_lenght=12, resampling=22050, original_length=60):
    record, sr = torchaudio.load(file)
    audio_len = audio_lenght*resampling
    record = torch.mean(record, dim=0, keepdim=True)
    record = torchaudio.transforms.Resample(sr, resampling)(record)
    missing_padding = resampling * original_length - record.shape[1]
    padding = torch.zeros([1, missing_padding])
    record = torch.cat((record, padding), axis=1)
    print(record.shape[1]/resampling, missing_padding)
    record = record[:, :audio_len*(record.shape[1] // audio_len)]
    record = torch.reshape(record, (record.shape[1] // audio_len, audio_len))
    print(record.shape)
    record = record[0]
    return record
    

In [ ]:
record = fileToRecord(file)